# CNN-RNN

In [1]:
import tensorflow.keras as keras

In [2]:
# Módulos necesarios. Asegurarse de poder importarlos.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from pylab import rcParams
from scipy import stats
from sklearn import metrics
import seaborn as sns
from sklearn import preprocessing
import os as os
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
from time import time
from tensorflow.keras.models import load_model

# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [3]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
tf.__version__

/usr/bin/python
3.6.8 (default, Aug 20 2019, 17:12:48) 
[GCC 8.3.0]
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)


'1.14.0'

In [4]:
keras.__version__

'2.2.4-tf'

# 12-FOLD

In [5]:
subjects = ['01', '02', '03', '05', '08', '09', '10', '11', '13', '14', '16', '17']
global activities
activities = [9,10,11,12,13,19,20,21,24,25,31]

In [6]:
def perf_measure(cm1):
    temp = 0
    TruePositive = np.diag(cm1)
    FalsePositive = []
    for i in range(len(activities)):
        FalsePositive.append(sum(cm1[:,i]) - cm1[i,i])
    FalseNegative = []
    for i in range(len(activities)):
        FalseNegative.append(sum(cm1[i,:]) - cm1[i,i])
    TrueNegative = []
    for i in range(len(activities)):
        temp = np.delete(cm1, i, 0)   # delete ith row
        temp = np.delete(temp, i, 1)  # delete ith column
        TrueNegative.append(sum(sum(temp)))
    
    return(TruePositive, FalsePositive, TrueNegative, FalseNegative)

In [7]:
# Podríamos variarlo
batch_size = 128 # Tamaño del batch
learning_rate = 1e-3 # Learning rate (por defecto es 0.001)

In [8]:
accuracies = np.zeros((12))
#accuraciesClass = np.zeros((12,11))
precision = np.zeros((12,11))
recall = np.zeros((12,11))
fScore = np.zeros((12,11))
support = np.zeros((12,11))
specifities = np.zeros((12,11))

classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
measure = ['precision', 'recall', 'f1-score', 'support']

for i in range(12):
    print('##################################################################################################################################')
    print('#####################################################  K-FOLD %d  #################################################################'%(i+1))
    
    subject_test = subjects[i]
    print('Test subject: ' + str(subject_test))
    
    ##  GENERACIÓN DATASET
    ts_seg = np.load('./augment_2/subject_'+subject_test+'_seg.npy')
    ts_lab = np.load('./augment_2/subject_'+subject_test+'_lab.npy')
    
    limit = 40
    ts_seg = ts_seg[:,:,:,:limit]
    ####################################################################################################################
    ## SHUFFLE DE DATOS
    np.random.seed(235)
    ts_seg = np.reshape(ts_seg[np.random.shuffle(np.arange(0,ts_seg.shape[0]))], (2375,1,128,limit))
    ts_lab = np.reshape(ts_lab[np.random.shuffle(np.arange(0,ts_seg.shape[0]))], (2375,11))
    
    print('Test dataset: ')
    print(ts_seg.shape, ts_lab.shape)
    ####################################################################################################################
    ## RED
    model = load_model('./03AUGFFT_SP_KFOLD_models/CNN_RNN_'+subject_test+'.h5')
    model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate),metrics=['categorical_accuracy'])
    ####################################################################################################################
    ## TEST
    predictions = model.predict(ts_seg, batch_size = batch_size, verbose = 0)
    
    y_pred = np.argmax(predictions,axis=-1)
    y_pred = np.expand_dims(y_pred,axis=-1)
    
    y_true = np.argmax(ts_lab,axis=-1)
    y_true = np.expand_dims(y_true,axis=-1)
    
    y_true_flat = np.ndarray.flatten(y_true)
    y_pred_flat = np.ndarray.flatten(y_pred)
    ####################################################################################################################
    ## METRICS        
    # CONFUSION MATRIX
    print('\n')
    print('Confusion matrix:')
    cm = metrics.confusion_matrix(y_true, y_pred)
    print(cm)
    
    print('\n')
    tp, fp, tn, fn = perf_measure(cm)
    tp, fp, tn, fn = np.asarray(tp), np.asarray(fp), np.asarray(tn), np.asarray(fn)
    print('True Positive: ', str(tp))
    print('False Positive: ', str(fp))
    print('True Negative: ', str(tn))
    print('False Negative: ', str(fn))
    
    # ACCURACY
    print('\n')
    acc = metrics.accuracy_score(y_true_flat,y_pred_flat)
    accuracies[i] = acc
    print('Total accuracy: ')
    print(acc)
    '''
    #accuraciesClass[i] = cm.diagonal()/cm.sum(axis=1)
    accuraciesClass[i] = (tp+tn)/(tp+tn+fp+fn)
    print('Accuracy per class: ')
    for j in range(len(activities)):
        print('%d'%(j)+'\t'+ str(accuraciesClass[i,j]))
    '''
    # CLASSIFICATION REPORT
    print('\n')
    print('Classification report:')
    reportString = metrics.classification_report(y_true_flat, y_pred_flat, digits=3)
    print(reportString)
    report = metrics.classification_report(y_true_flat, y_pred_flat, digits=3, output_dict=True)
    
    for j in range(len(classes)):
        precision[i,j] = report[classes[j]][measure[0]]
        recall[i,j] = report[classes[j]][measure[1]]
        fScore[i,j] = report[classes[j]][measure[2]]
        support[i,j] = report[classes[j]][measure[3]]
    
    
    # SPECIFITY
    print('\n')
    print('Specifity: ')
    specifity = tn/(tn+fp)
    for j in range(len(activities)):
        print('%d'%(j)+'\t'+ str(specifity[j]))
        specifities[i,j] = specifity[j]
    
    fo = open('./03AUGFFT_SP_KFOLD_models/metrics_'+subject_test+'.txt', "w")
    fo.seek(0,2)
    fo.write('Accuracy: ' + str(acc))
    fo.write('\n')
    fo.write('Confusion matrix: ')
    fo.write('\n')
    fo.write(str(cm))
    fo.write('\n')
    fo.write('Specifity:')
    fo.write(str(specifity))
    fo.write('\n')
    fo.write('Report: ')
    fo.write('\n')
    fo.write(reportString)
    fo.close()
    
    
    del subject_test, ts_seg, ts_lab, model, y_pred, y_true, predictions, y_true_flat, y_pred_flat, reportString, specifity, cm, report
    

##################################################################################################################################
#####################################################  K-FOLD 1  #################################################################
Test subject: 01
Test dataset: 
(2375, 1, 128, 40) (2375, 11)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as n

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




Confusion matrix:
[[216   0   0   0   0   0   0   0   0   0   0]
 [  0 216   0   0   0   0   0   0   0   0   0]
 [  0   0 216   0   0   0   0   0   0   0   0]
 [  0   0   0 213   0   0   0   0   0   3   0]
 [  0   0   0   0 216   0   0   0   0   0   0]
 [  0   0   0   0   0 216   0   0   0   0   0]
 [  0   0   0   0   0   0 216   0   0   0   0]
 [  0   0   0   0   0   0   0 216   0   0   0]
 [  0   0   0   0   0   0   0   0 215   0   1]
 [  0   0   0   3   0   0   0   0 148  65   0]
 [  0   0   0   0   0   0   0   0   1   0 214]]


True Positive:  [216 216 216 213 216 216 216 216 215  65 214]
False Positive:  [  0   0   0   3   0   0   0   0 149   3   1]
True Negative:  [2159 2159 2159 2156 2159 2159 2159 2159 2010 2156 2159]
False Negative:  [  0   0   0   3   0   0   0   0   1 151   1]


Total accuracy: 
0.9343157894736842


Classification report:
              precision    recall  f1-score   support

           0      1.000     1.000     1.000       216
           1      1.000    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




Confusion matrix:
[[216   0   0   0   0   0   0   0   0   0   0]
 [149  67   0   0   0   0   0   0   0   0   0]
 [  0   0 216   0   0   0   0   0   0   0   0]
 [  0   0   0  83   0   0 133   0   0   0   0]
 [  0   0   5   0 211   0   0   0   0   0   0]
 [  0   0   0   0   0 216   0   0   0   0   0]
 [  0   0   0   0   0   0 216   0   0   0   0]
 [  0   0   0   0   0   3   0 213   0   0   0]
 [  0   0   0   0   0   0   0   0 211   0   5]
 [  0   0   0   0   0  13   0   0   0 203   0]
 [  0   0   0   0   0   0   0   0  21   0 194]]


True Positive:  [216  67 216  83 211 216 216 213 211 203 194]
False Positive:  [149   0   5   0   0  16 133   0  21   0   5]
True Negative:  [2010 2159 2154 2159 2159 2143 2026 2159 2138 2159 2155]
False Negative:  [  0 149   0 133   5   0   0   3   5  13  21]


Total accuracy: 
0.8614736842105263


Classification report:
              precision    recall  f1-score   support

           0      0.592     1.000     0.744       216
           1      1.000    

In [9]:
'''
acc = np.sum(accuraciesClass,axis=0)/12
print('Accuracy per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(acc[j]))
print('Total accuracy: ' + str(sum(acc)/11))
'''
print('Total accuracy: ' + str(sum(accuracies)/12))

prec = np.sum(precision,axis=0)/12
print('Precision per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(prec[j]))
print('Total precision: ' + str(sum(prec)/11))

rec = np.sum(recall,axis=0)/12
print('Recall per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(rec[j]))
print('Total recall: ' + str(sum(rec)/11))

fS = np.sum(fScore,axis=0)/12
print('F1-score per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(fS[j]))
print('Total F1-score: ' + str(sum(fS)/11))

spec = np.sum(specifities,axis=0)/12
print('Specifities per class: ')
for j in range(11):
    print('%d'%(j)+'\t'+str(spec[j]))
print('Total specifities: ' + str(sum(spec)/11))

Total accuracy: 0.9350877192982457
Precision per class: 
0	0.9476303512849679
1	0.9981146304675717
2	0.981055617985927
3	0.9672497149257259
4	0.9898373983739838
5	0.9423025044221799
6	0.8627820995623182
7	0.9550716337849572
8	0.894989010556441
9	0.8821648333061659
10	0.9213699754105352
Total precision: 0.9402334336437068
Recall per class: 
0	1.0
1	0.9232253086419754
2	0.9984567901234568
3	0.91820987654321
4	0.9560185185185185
5	0.9228395061728397
6	0.8464506172839507
7	0.976466049382716
8	0.9432870370370369
9	0.8418209876543209
10	0.9593023255813953
Total recall: 0.9350979106308565
F1-score per class: 
0	0.9683177799578493
1	0.9449071692188554
2	0.9891363412936743
3	0.9294144141953006
4	0.9662143800488464
5	0.9174626933683391
6	0.8414223977394601
7	0.9612451439745003
8	0.9020219232573582
9	0.8458735386355979
10	0.9279330762285029
Total F1-score: 0.9267226234471168
Specifities per class: 
0	0.9918943955534969
1	0.9998070094179404
2	0.9978385054809326
3	0.9962173845916319
4	0.99884205650